In [23]:
# Assuming the file name is movies.csv
import pandas as pd
df = pd.read_csv(r"C:\Users\saket\Downloads\movielens-ratings.csv")


# for i in df['title']:
#     print('site:trakt.tv '+i)


,movie_id,imdb_id,tmdb_id,rating,average_rating,title
0,1,114709,862,4.0,3.89314,Toy Story (1995)
1,32,114746,63,4.0,3.89616,Twelve Monkeys (1995)
2,47,114369,807,4.5,4.08738,Se7en (1995)
3,50,114814,629,4.5,4.26569,The Usual Suspects (1995)
4,111,75314,103,4.5,4.08057,Taxi Driver (1976)
...,...,...,...,...,...,...
593,279520,21929356,1005031,4.0,3.95000,A Trip to Infinity (2022)
594,281190,10640346,615777,3.0,3.37147,Babylon (2022)
595,281982,22890438,1040533,3.0,2.90000,Trevor Noah: I Wish You Would (2022)
596,282645,24224996,1066719,3.0,2.66667,Mumbai Mafia: Police vs the Underworld (2023)


# chrome working

In [2]:

# Import the selenium webdriver module
from selenium import webdriver
# import the ChromeOptions class
from selenium.webdriver.chrome.options import Options as chromeOptions
import time

# Create a ChromeOptions object
options = webdriver.ChromeOptions()

# Use the default profile name in the user-data-dir argument
options.add_argument("--user-data-dir=C:\\Users\\saket\\AppData\\Local\\Google\\Chrome\\User Data")

# options.add_argument("--user-data-dir=C:\\Temp\\ChromeProfile")

# Create a webdriver object for chrome with the options
driver = webdriver.Chrome(options=options)

# Open the google.com website
driver.get("https://trakt.tv/dashboard")

# Get the title of the page

print(driver.title)

time.sleep(60000)

# Close the driver
# driver.close()


WebDriverException: Message: unknown error: cannot determine loading status
from no such window
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x010DA813+48355]
	(No symbol) [0x0106C4B1]
	(No symbol) [0x00F75220]
	(No symbol) [0x00F688E2]
	(No symbol) [0x00F67138]
	(No symbol) [0x00F677AA]
	(No symbol) [0x00F708A9]
	(No symbol) [0x00F7C668]
	(No symbol) [0x00F7F566]
	(No symbol) [0x00F67BC3]
	(No symbol) [0x00F7C37A]
	(No symbol) [0x00FCCB1F]
	(No symbol) [0x00FBA536]
	(No symbol) [0x00F982DC]
	(No symbol) [0x00F993DD]
	GetHandleVerifier [0x0133AABD+2539405]
	GetHandleVerifier [0x0137A78F+2800735]
	GetHandleVerifier [0x0137456C+2775612]
	GetHandleVerifier [0x011651E0+616112]
	(No symbol) [0x01075F8C]
	(No symbol) [0x01072328]
	(No symbol) [0x0107240B]
	(No symbol) [0x01064FF7]
	BaseThreadInitThunk [0x75177D59+25]
	RtlInitializeExceptionChain [0x771EB79B+107]
	RtlClearBits [0x771EB71F+191]


In [ ]:
# Import the selenium webdriver module
from selenium import webdriver

# Create a ChromeOptions object
options = webdriver.ChromeOptions()

# Use the default profile name in the user-data-dir argument
options.add_argument("--user-data-dir=Default")

# Create a webdriver object for chrome with the options
driver = webdriver.Chrome(options=options)

# Open the google.com website
driver.get("https://www.google.com")

# Close the driver
driver.close()


In [ ]:
# Import the selenium webdriver module
from selenium import webdriver

# Create a ChromeOptions object
options = webdriver.ChromeOptions()

# Get the path of your default profile
# profile_path = "C:\\Users\\pc\\AppData\\Local\\Google\\Chrome\\User Data"
# profile_path = r"C:\Users\pc\AppData\Local\Google\Chrome\User Data"


options = webdriver.ChromeOptions()
options.add_argument(f"user-data-dir=Default")

# Create a webdriver object for chrome with the options
driver = webdriver.Chrome(options=options)

# Open the google.com website
driver.get("https://www.google.com")

# Close the driver
driver.close()


In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options

# service = Service(executable_path=r"C:\path\edge\msedgedriver.exe")

options = Options()
options.add_argument("user-data-dir=C:\\Users\\saket\\AppData\\Local\\Microsoft\\Edge\\User Data")
# options.add_argument("--remote-debugging-port=9222")
options.binary_location = r"C:\Program Files\Microsoft Edge\Application\msedge.exe"
driver = webdriver.Edge(options=options)

driver.get("https://www.google.com/")
# Do some actions on the Google homepage

driver.close()


In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.edge.service import Service

options = Options()
#Here you set the path of the profile ending with User Data not the profile folder
options.add_argument("user-data-dir=C:\\Users\\saket\\AppData\\Local\\Microsoft\\Edge\\User Data\\profile")
# Here you specify the actual profile folder
options.add_argument("profile-directory=Default")
#Here you add the arguments for debugging and memory usage
options.add_argument("--remote-debugging-port=9222")
options.add_argument("--disable-dev-shm-usage")
# Here you set the binary location of the Edge Dev application 
options.binary_location = r"C:\Program Files (x86)\Microsoft\Edge Dev\Application\msedge.exe"

service = Service(r"C:\path\edge\msedgedriver.exe")
# service = Service(r"C:\path\msedgedriver.exe")
driver = webdriver.Edge(service=service, options=options)
driver.get('https://google.com')
driver.quit()


# worked

In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
import selenium
import pickle

# service = Service(executable_path=r"C:\path\edge\msedgedriver.exe")

options = Options()

# Save cookies
driver = selenium.webdriver.Edge(options=options)
driver.get("https://www.google.com")
pickle.dump(driver.get_cookies(), open("cookies.pkl", "wb"))

# Load cookies
driver = selenium.webdriver.Edge(options=options)
driver.get("https://www.google.com")
cookies = pickle.load(open("cookies.pkl", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)


In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.edge.service import Service

options = Options()
#Here you set the path of the profile ending with User Data not the profile folder
options.add_argument("user-data-dir=C:\\Users\\saket\\AppData\\Local\\Microsoft\\Edge\\User Data")
#Here you specify the actual profile folder
options.add_argument("profile-directory=Default")
    
service = Service(r"C:\path\edge\msedgedriver.exe")
driver = webdriver.Edge(service=service, options=options)
driver.get('https://googleclear.com')
driver.quit()


# worked

In [ ]:

from msedge.selenium_tools import Edge, EdgeOptions
options = EdgeOptions()
options.add_argument("C:/Users/saket/AppData/Local/Microsoft/Edge/")
driver = Edge(executable_path="C:/path/edge/msedgedriver.exe", options=options)
driver.get("https://www.example.com")
driver.add_cookie({"name": "foo", "value": "bar"})


In [14]:
# Import selenium webdriver
from selenium import webdriver
from webdriver_manager.microsoft import EdgeChromiumDriverManager

# Create a driver object with the EdgeChromiumDriverManager
driver = webdriver.Edge(EdgeChromiumDriverManager().install())

# Navigate to www.google.com
driver.get("https://www.google.com")

# Close the driver
driver.quit()


NoSuchDriverException: Message: Unable to obtain msedgedriver using Selenium Manager; 'str' object has no attribute 'capabilities'; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [5]:
```python
from selenium import webdriver
from msedge.selenium_tools import EdgeOptions

# Set the profile and user data directories
options = EdgeOptions()
options.use_chromium = True
options.binary_location = "C:\\Program Files (x86)\\Microsoft\\Edge\\Application\\msedge.exe" # Change this to your desired path
options.add_argument("user-data-dir=C:\\Users\\saket\\AppData\\Local\\Microsoft\\Edge\\User Data")
options.add_argument("profile-directory=Default")

# Launch edge with the options
driver = webdriver.Edge(options=options)

# Navigate to a website that uses cookies
driver.get("https://www.google.com")
```

AttributeError                            Traceback (most recent call last)
File c:\Users\saket\anaconda3\lib\site-packages\selenium\webdriver\common\driver_finder.py:42, in DriverFinder.get_path(service, options)
     41 try:
---> 42     path = SeleniumManager().driver_location(options) if path is None else path
     43 except Exception as err:

File c:\Users\saket\anaconda3\lib\site-packages\selenium\webdriver\common\selenium_manager.py:78, in SeleniumManager.driver_location(self, options)
     76 args = [str(self.get_binary()), "--browser", browser, "--output", "json"]
---> 78 if options.browser_version:
     79     args.append("--browser-version")

AttributeError: 'Options' object has no attribute 'browser_version'

During handling of the above exception, another exception occurred:

NoSuchDriverException                     Traceback (most recent call last)
c:\Users\saket\Documents\GitHub\Pyhton\web scraping\selenium\test.ipynb Cell 1 in ()
      9 options.add_argument("profile-directory=Default")
     11 # Launch edge with the options
---> 12 driver = webdriver.Edge(options=options)
     14 # Navigate to a website that uses cookies
     15 driver.get("https://www.google.com")

File c:\Users\saket\anaconda3\lib\site-packages\selenium\webdriver\edge\webdriver.py:47, in WebDriver.__init__(self, options, service, keep_alive)
     44 self.options = options if options else Options()
     45 self.keep_alive = keep_alive
---> 47 self.service.path = DriverFinder.get_path(self.service, self.options)
     49 super().__init__(
     50     DesiredCapabilities.EDGE["browserName"],
     51     "ms",
   (...)
     54     self.keep_alive,
     55 )

File c:\Users\saket\anaconda3\lib\site-packages\selenium\webdriver\common\driver_finder.py:44, in DriverFinder.get_path(service, options)
     42     path = SeleniumManager().driver_location(options) if path is None else path
     43 except Exception as err:
---> 44     raise NoSuchDriverException(f"Unable to obtain {service.path} using Selenium Manager; {err}")
     46 if path is None or not Path(path).is_file():
     47     raise NoSuchDriverException(f"Unable to locate or obtain {service.path}")

NoSuchDriverException: Message: Unable to obtain msedgedriver using Selenium Manager; 'Options' object has no attribute 'browser_version'; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location

NoSuchDriverException: Message: Unable to obtain msedgedriver using Selenium Manager; 'Options' object has no attribute 'browser_version'; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
